## Baseline Bert
This notebook implments a baseline model, the comment binary classification

In [ ]:
!pip install torchmetrics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 14.6 MB/s eta 0:00:00


In [ ]:
import os
import csv
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torchmetrics import AUROC, F1Score

In [ ]:
torch.manual_seed(0)

In [3]:
import pandas as pd
from google.colab import drive
drive = drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/Colab_Notebooks/deep/toxic/scr/kaggle_data/'
df_x = pd.read_csv(data_dir+'train_x.csv')  #  index, string
df_y = pd.read_csv(data_dir+'train_y.csv')  #  index, anothers和 y

# merge by index
df = pd.merge(df_x, df_y,left_index=True, right_index=True)

none_rows = df[df['string'].isnull()]
# print(df.head())


Mounted at /content/drive


In [4]:
print(none_rows)

df = df.dropna(subset=['string'])

print(f"row number : {len(df)}")

         index string  male  female  LGBTQ  christian  muslim  \
141037  141037    NaN     0       0      0          0       0   

        other_religions  black  white  identity_any  severe_toxicity  obscene  \
141037                0      0      0             0                0        0   

        threat  insult  identity_attack  sexual_explicit  y  \
141037       0       0                0                0  0   

        from_source_domain  
141037                   1  
row number : 269037


In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print("Train size:", len(train_df))
print("Validation size:", len(val_df))


Train size: 242133
Validation size: 26904


In [6]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
from datasets import Dataset
from transformers import AutoTokenizer


model_name = "bert-base-uncased"


tokenizer = AutoTokenizer.from_pretrained(model_name)

# switch pandas DataFrame to HuggingFace Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)


def tokenize_function(example):
    return tokenizer(
        example["string"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# apply tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# set Dataset to (Tensor)
train_dataset = train_dataset.rename_column("y", "labels")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

val_dataset = val_dataset.rename_column("y", "labels")
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/242133 [00:00<?, ? examples/s]

Map:   0%|          | 0/26904 [00:00<?, ? examples/s]

In [8]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.3 MB/s eta 0:00:00


In [9]:
import torch
from transformers import BertForSequenceClassification, TrainingArguments, Trainer


model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)


import numpy as np
from evaluate import load

accuracy_metric = load("accuracy")


f1_metric = load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)

    return {
        "accuracy": acc["accuracy"],

    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-9-9e94da69dda6>:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.267700,0.227666,0.924509
2,0.213300,0.286186,0.925885


Epoch,Training Loss,Validation Loss,Accuracy
1,0.267700,0.227666,0.924509
2,0.213300,0.286186,0.925885
3,0.187500,0.315041,0.924881


TrainOutput(global_step=90801, training_loss=0.21876377265967525, metrics={'train_runtime': 5236.0964, 'train_samples_per_second': 138.729, 'train_steps_per_second': 17.341, 'total_flos': 4.778090187561216e+16, 'train_loss': 0.21876377265967525, 'epoch': 3.0})

In [10]:
df_test = pd.read_csv(data_dir+'test_x.csv')

In [11]:
df_test.head()

,index,string
0,0,OH yes - Were those evil Christian Missionarie...
1,1,He's considered a good candidate for a cyber-s...
2,2,"Lela, you admit no records exist to support yo..."
3,3,I'll take the iffy libertarian over the guy wh...
4,4,Shouldn't your handle be Republic of Uranus?


In [12]:
test_inputs =df_test.loc[:,'string'].fillna('').tolist()

In [13]:
import torch
from transformers import BertForSequenceClassification, AutoTokenizer

model_path = "./results/checkpoint-90801"


model = BertForSequenceClassification.from_pretrained(model_path)



tokenizer = AutoTokenizer.from_pretrained(model_path)


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = tokenizer(test_inputs, return_tensors="pt", truncation=True, max_length=128,padding=True).to(device)

In [16]:
batch_size = 32
predictions = []
model.eval()
model.to(device)
for i in range(0, len(inputs['input_ids']), batch_size):
    batch_inputs = {key: value[i:i+batch_size].to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**batch_inputs)
        logits = outputs.logits
        batch_preds = torch.argmax(logits, dim=-1).tolist()
        predictions.extend(batch_preds)

print('finish the collection of predictions', predictions)


finish the collection of predictions [1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,

ValueError: If using all scalar values, you must pass an index

In [17]:
pred_df = pd.DataFrame({'ID': range(len(predictions)), 'pred': predictions})

In [19]:
pred_df.to_csv('prediction.csv', index=False)
print('finish the writing of predictions to csv')

finish the writing of predictions to csv
